<a href="https://colab.research.google.com/github/hassanfv/hfv_GPU/blob/main/div_curlV_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile test.cu
#include <iostream> // include iostream, fstream, cmath.
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
using namespace std;

const int N = 131504;
const float my_pi = 3.141592f;
//const float visc_alpha = 1.0f;

__global__ void div_curlVel(float *divV, float *curlV, float *x, float *y, float *z, float *vx, float *vy, float *vz,
                            float *rho, float *mass, float *h){
  int i = threadIdx.x + blockIdx.x * blockDim.x;

  if(i < N){

    float dx, dy, dz, rr, hij, q, vxji, vyji, vzji, hij5, sig;
    float nW = 0.0f; float gWx = 0.0f; float gWy = 0.0f; float gWz = 0.0f;
    float vxij, vyij, vzij;
    float ss = 0.0f; float curlVx = 0.0f; float curlVy = 0.0f; float curlVz = 0.0f;

    for(int j = 0; j < N; j++){
      dx = x[j] - x[i];
      dy = y[j] - y[i];
      dz = z[j] - z[i];

      rr = sqrt(dx*dx + dy*dy + dz*dz);
      hij = 0.5f * (h[i] + h[j]);
      q =rr/hij;

      if(q <= 2.0f){

        nW = 0.0f; gWx = 0.0f; gWy = 0.0f; gWz = 0.0f;

        sig = 1.0f/my_pi;
        hij5 = hij*hij*hij*hij*hij;

        if(q <= 1.0f){
          nW = sig/hij5 * (-3.0f + (9.0f/4.0f) * q);
          gWx = nW * dx;
          gWy = nW * dy;
          gWz = nW * dz;
        }

        if((q > 1.0f) && (q <=2.0f)){
          nW = -3.0f*sig/(4.0f*hij5) * (2.0f - q)*(2.0f - q) / (q+1e-10);
          gWx = nW * dx;
          gWy = nW * dy;
          gWz = nW * dz;
        }

        vxji = vx[j] - vx[i];
        vyji = vy[j] - vy[i];
        vzji = vz[j] - vz[i];

        ss += mass[j]/rho[i] * (vxji*gWx + vyji*gWy + vzji*gWz);

        vxij = vx[i] - vx[j]; //-vxji;
        vyij = vy[i] - vy[j]; //-vyji;
        vzij = vz[i] - vz[j]; //-vzji;

        curlVx += mass[j]/rho[i] * (vyij*gWz - vzij*gWy);
        curlVy += mass[j]/rho[i] * (vzij*gWx - vxij*gWz);
        curlVz += mass[j]/rho[i] * (vxij*gWy - vyij*gWx);
      }
    }
    divV[i] = abs(ss);
    curlV[i] = sqrt(curlVx*curlVx + curlVy*curlVy + curlVz*curlVz);
  }
}

int main(){

  // Reading Hydra file.
  string fname = "Hydra_130k.csv";

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  float *x,*y,*z, *vx,*vy,*vz, *rho, *P, *c, *h, *mass, *divV, *curlV;
  float *d_x,*d_y,*d_z, *d_vx,*d_vy,*d_vz, *d_rho, *d_P, *d_c, *d_h, *d_mass, *d_divV, *d_curlV;

  x = new float[N];
  y = new float[N];
  z = new float[N];

  vx = new float[N];
  vy = new float[N];
  vz = new float[N];

  rho = new float[N];
  P = new float[N];
  c = new float[N];
  h = new float[N];
  mass = new float[N];
  divV = new float[N];
  curlV = new float[N];

  // 0  1  2  3   4   5    6   7  8  9  10
  // x, y, z, vx, vy, vz, rho, P, c, h, m.

  for(int i=0; i<N; i++){

    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);

    vx[i] = stof(content[i][3]);
    vy[i] = stof(content[i][4]);
    vz[i] = stof(content[i][5]);

    rho[i] = stof(content[i][6]);
    P[i] = stof(content[i][7]);
    c[i] = stof(content[i][8]);
    h[i] = stof(content[i][9]);
    mass[i] = stof(content[i][10]);

    divV[i] = 110.0f;
    curlV[i] = 110.0f;

  }

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));
  cudaMalloc(&d_z, N*sizeof(float));

  cudaMalloc(&d_vx, N*sizeof(float));
  cudaMalloc(&d_vy, N*sizeof(float));
  cudaMalloc(&d_vz, N*sizeof(float));

  cudaMalloc(&d_rho, N*sizeof(float));
  cudaMalloc(&d_P, N*sizeof(float));
  cudaMalloc(&d_c, N*sizeof(float));
  cudaMalloc(&d_h, N*sizeof(float));
  cudaMalloc(&d_mass, N*sizeof(float));
  cudaMalloc(&d_divV, N*sizeof(float));
  cudaMalloc(&d_curlV, N*sizeof(float));

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_vx, vx, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vy, vy, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vz, vz, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_rho, rho, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_P, P, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_c, c, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_h, h, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_mass, mass, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_divV, divV, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_curlV, curlV, N*sizeof(float), cudaMemcpyHostToDevice);


  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  div_curlVel<<<gridSize, blockSize>>>(d_divV, d_curlV, d_x, d_y, d_z, d_vx, d_vy, d_vz,
                                       d_rho, d_mass, d_h);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();

  // Copy from Device to Host.
  cudaMemcpy(divV, d_divV, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(curlV, d_curlV, N*sizeof(float), cudaMemcpyDeviceToHost);


  // Output to a file
  ofstream outfile("div_curlV_from_cpp_130k.csv");
  if(outfile.is_open()){
    for(int i = 0; i < N; i++){
      outfile << divV[i] << "," << curlV[i] << endl;
    }
  }else cout << "Unable to open file !";


  // visual check.
  for(int i = 0; i < 10; i++){
    cout << divV[i] << " " << curlV[i] << endl;
    //cout << mass[i] << " " << vx[i] << " " << h[i] << endl;
  }

  // Free memory.
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);

  cudaFree(d_vx);
  cudaFree(d_vy);
  cudaFree(d_vz);

  cudaFree(d_rho);
  cudaFree(d_P);
  cudaFree(d_c);
  cudaFree(d_h);
  cudaFree(d_mass);
  cudaFree(d_divV);
  cudaFree(d_curlV);

  delete[] x;
  delete[] y;
  delete[] z;

  delete[] vx;
  delete[] vy;
  delete[] vz;

  delete[] rho;
  delete[] P;
  delete[] c;
  delete[] h;
  delete[] mass;
  delete[] divV;
  delete[] curlV;

}

Writing test.cu


In [2]:
%%shell
nvcc test.cu -o test

In [3]:
%%shell
./test

0.0960596 0.0724935
0.531285 0.26442
0.546801 0.191839
0.675974 0.213807
0.622348 0.363959
0.365447 0.0924436
0.537905 0.149043
0.846763 0.0581866
0.55048 0.110455
0.117956 0.13458


In [4]:
%%shell
nvprof ./test

==195== NVPROF is profiling process 195, command: ./test
0.0960596 0.0724935
0.531285 0.26442
0.546801 0.191839
0.675974 0.213807
0.622348 0.363959
0.365447 0.0924436
0.537905 0.149043
0.846763 0.0581866
0.55048 0.110455
0.117956 0.13458
==195== Profiling application: ./test
==195== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.82%  381.24ms         1  381.24ms  381.24ms  381.24ms  div_curlVel(float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*)
                    0.15%  589.40us        13  45.338us  45.055us  45.663us  [CUDA memcpy HtoD]
                    0.02%  83.263us         2  41.631us  41.600us  41.663us  [CUDA memcpy DtoH]
      API calls:   53.60%  381.26ms         1  381.26ms  381.26ms  381.26ms  cudaDeviceSynchronize
                   45.87%  326.28ms        13  25.098ms  2.7600us  325.86ms  cudaMalloc
                    0.36%  2.5267ms        15  168.45us  136